In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from PIL import Image

def build_dehazenet_model(target_size, num_channels):
    model = models.Sequential()
    # Define your DehazeNet model architecture
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(target_size, target_size, num_channels)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    # Upsampling layers
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.Conv2D(num_channels, (3, 3), activation='sigmoid', padding='same'))  # Output with 3 channels (RGB)
    
    return model



def preprocess_image(image, target_size):
    # Resize the image to the target size
    resized_image = image.resize((target_size, target_size))

    # Convert the image to grayscale if it has an alpha channel
    if resized_image.mode == 'RGBA':
        resized_image = resized_image.convert('RGB')

    # Convert the image to numpy array
    image_array = np.array(resized_image)

    # If the image has only 2 dimensions, expand it to have a third dimension
    if len(image_array.shape) == 2:
        image_array = np.expand_dims(image_array, axis=2)

    return image_array



def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            if image is not None:
                images.append(image)
    return images


def preprocess_images(images, target_size):
    preprocessed_images = []
    for image in images:
        preprocessed_image = preprocess_image(image, target_size)
        preprocessed_images.append(preprocessed_image)

    # Find the maximum shape among all preprocessed images
    max_shape = max([image.shape for image in preprocessed_images])

    # Resize all preprocessed images to the maximum shape
    resized_images = []
    for image in preprocessed_images:
        resized_image = np.pad(image, ((0, max_shape[0] - image.shape[0]),
                                       (0, max_shape[1] - image.shape[1]),
                                       (0, max_shape[2] - image.shape[2])),
                               mode='constant')
        resized_images.append(resized_image)

    # Convert the list of preprocessed images to a NumPy array
    preprocessed_images = np.array(resized_images)
    return preprocessed_images




def supervised_train(foggy_images, clear_images, num_epochs, batch_size):
    # Resize clear images to match the size of foggy images
    resized_clear_images = []
    target_size = foggy_images.shape[1]
    for image in clear_images:
        resized_image = np.array(Image.fromarray(image).resize((target_size, target_size)))
        resized_clear_images.append(resized_image)

    # Convert lists to numpy arrays
    foggy_images = np.array(foggy_images)
    clear_images = np.array(resized_clear_images)

    # Build and compile the DehazeNet model
    num_channels = foggy_images.shape[3]
    model = build_dehazenet_model(target_size, num_channels)
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the DehazeNet model using supervised learning
    model.fit(foggy_images, clear_images, epochs=num_epochs, batch_size=batch_size)

    return model




if __name__ == '__main__':
    foggy_folder_path = "C:/Users/HP/Pictures/clearFoggy/Foggy/"  # Specify the folder path containing the foggy images
    clear_folder_path = "C:/Users/HP/Pictures/clearFoggy/Clear/"  # Specify the folder path containing the clear images

    foggy_images = load_images_from_folder(foggy_folder_path)
    clear_images = load_images_from_folder(clear_folder_path)

    print("Number of foggy images:", len(foggy_images))
    print("Number of clear images:", len(clear_images))

    target_size = 32  # Specify the desired target size for preprocessing

    preprocessed_foggy_images = preprocess_images(foggy_images, target_size)
    preprocessed_clear_images = preprocess_images(clear_images, target_size)

    print("Preprocessed foggy images shape:", preprocessed_foggy_images.shape)
    print("Preprocessed clear images shape:", preprocessed_clear_images.shape)

    # Normalize pixel values to [0, 1]
    preprocessed_foggy_images = preprocessed_foggy_images.astype('float32') / 255.0
    preprocessed_clear_images = preprocessed_clear_images.astype('float32') / 255.0

    # Train the DehazeNet using supervised learning
    model = supervised_train(preprocessed_foggy_images, preprocessed_clear_images, num_epochs=10, batch_size=30)

    # Now you can use the trained model for fog removal on new foggy images
    # Assuming you have a new foggy image: foggy_image_path
    foggy_image_path = "C:/Users/HP/Documents/Semester (summer)/Aim Lab Internship/Fog/foggy-001.jpg"
    foggy_image = Image.open(foggy_image_path)
    preprocessed_foggy_image = preprocess_image(foggy_image, target_size)
    normalized_foggy_image = preprocessed_foggy_image.astype('float32') / 255.0
    dehazed_image = model.predict(np.expand_dims(normalized_foggy_image, axis=0))[0]

    # Display the dehazed image
    plt.imshow(dehazed_image)
    plt.show()


Number of foggy images: 10
Number of clear images: 10
Preprocessed foggy images shape: (10, 32, 32, 3)
Preprocessed clear images shape: (10, 32, 32, 3)


TypeError: Cannot handle this data type: (1, 1, 3), <f4

In [14]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from PIL import Image


def build_dehazenet_model(target_size, num_channels):
    model = models.Sequential()
    # Define your DehazeNet model architecture
    
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(target_size, target_size, num_channels)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    # Add more convolutional layers as needed
    
    # Upsampling layers
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.Conv2D(num_channels, (3, 3), activation='sigmoid', padding='same'))  # Output with 3 channels (RGB)
    
    return model


def preprocess_image(image, target_size):
    # Resize the image to the target size
    resized_image = image.resize(( target_size, target_size))

    # Convert the image to grayscale if it has an alpha channel
    if resized_image.mode == 'RGBA':
        resized_image = resized_image.convert('RGB')

    # Convert the image to numpy array
    image_array = np.array(resized_image)

    # If the image has only 2 dimensions, expand it to have a third dimension
    if len(image_array.shape) == 2:
        image_array = np.expand_dims(image_array, axis=2)

    return image_array


def load_images_from_folder(folder_path, target_size):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            if image is not None:
                preprocessed_image = preprocess_image(image, target_size)
                images.append(preprocessed_image)
    return images


def preprocess_images(images, target_size):
    preprocessed_images = []
    for image in images:
        preprocessed_image = preprocess_image(image, target_size)
        preprocessed_images.append(preprocessed_image.copy())  # Create a copy of the preprocessed image
                                                               # to avoid modifying the original image array

    # Find the maximum shape among all preprocessed images
    max_shape = max([image.shape for image in preprocessed_images])

    # Resize all preprocessed images to the maximum shape
    resized_images = []
    for image in preprocessed_images:
        resized_image = np.pad(image, ((0, max_shape[0] - image.shape[0]),
                                       (0, max_shape[1] - image.shape[1]),
                                       (0, max_shape[2] - image.shape[2])),
                               mode='constant')
        resized_images.append(resized_image)

    # Convert the list of preprocessed images to a NumPy array
    preprocessed_images = np.array(resized_images)
    return preprocessed_images


def supervised_train(foggy_images, clear_images, num_epochs, batch_size, target_size):
    # Resize clear images to match the size of foggy images
    resized_clear_images = []
    for image in clear_images:
        resized_image = np.array(Image.fromarray(image).resize((target_size, target_size)))
        resized_clear_images.append(resized_image)

    # Convert lists to numpy arrays
    foggy_images = np.array(foggy_images)
    clear_images = np.array(resized_clear_images)

    # Build and compile the DehazeNet model
    num_channels = foggy_images.shape[3]
    model = build_dehazenet_model(target_size, num_channels)
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the DehazeNet model using supervised learning
    model.fit(foggy_images, clear_images, epochs=num_epochs, batch_size=batch_size)

    return model


if __name__ == '__main__':
    foggy_folder_path = "C:/Users/HP/Pictures/clearFoggy/Foggy"  # Specify the folder path containing the foggy images
    clear_folder_path = "C:/Users/HP/Pictures/clearFoggy/Clear"  # Specify the folder path containing the clear images

    target_size = 32  # Specify the desired target size for preprocessing

    foggy_images = load_images_from_folder(foggy_folder_path, target_size)
    clear_images = load_images_from_folder(clear_folder_path, target_size)

    print("Number of foggy images:", len(foggy_images))
    print("Number of clear images:",len(clear_images))

    # Preprocess the images
    foggy_images = preprocess_images(foggy_images, target_size)
    clear_images = preprocess_images(clear_images, target_size)

    print("Shape of preprocessed foggy images:", foggy_images.shape)
    print("Shape of preprocessed clear images:", clear_images.shape)

    # Train the DehazeNet model
    num_epochs = 10
    batch_size = 32
    model = supervised_train(foggy_images, clear_images, num_epochs, batch_size, target_size)

    # Evaluate the DehazeNet model on a test image (optional)
    test_image_path = "C:/Users/HP/Documents/Semester (summer)/Aim Lab Internship/Fog/foggy-001.jpg"
    test_image = Image.open(test_image_path)
    test_foggy_image = preprocess_image(test_image, target_size)
    test_foggy_image = np.expand_dims(test_foggy_image, axis=0)
    predicted_clear_image = model.predict(test_foggy_image)
    plt.imshow(test_foggy_image[0])
    plt.show()
    plt.imshow(predicted_clear_image[0])
    plt.show()

Number of foggy images: 10
Number of clear images: 10


ValueError: cannot resize an array that references or is referenced
by another array in this way.
Use the np.resize function or refcheck=False

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow.keras import layers, models, callbacks
from PIL import Image

import os
import glob

from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow as tf


def build_dehazenet_model(target_size, num_channels):
    """
    Build the DehazeNet model architecture.

    Parameters:
        target_size (int): the size of the input images (assumed to be square).
        num_channels (int): the number of color channels in the input images (usually 3 for RGB).

    Returns:
        a Keras model object.
    """
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(target_size, target_size, num_channels)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    # Add more convolutional layers as needed

    # Upsampling layers
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(layers.Conv2D(num_channels, (3, 3), activation='sigmoid', padding='same'))  # Output with 3 channels (RGB)

    return model


def preprocess_image(image, target_size):
    """
    Preprocess an image by resizing it to the target size and converting it to a NumPy array.

    Parameters:
        image (PIL.Image): the input image.
        target_size (int): the size to which the image should be resized (assumed to be square).

    Returns:
        a NumPy array representing the preprocessed image.
    """
    # Resize the image to the target size
    resized_image = image.resize((target_size, target_size))

    # Convert the image to grayscale if it has an alpha channel
    if resized_image.mode == 'RGBA':
        resized_image = resized_image.convert('RGB')

    # Convert the image to numpy array
    image_array = np.array(resized_image)

    # If the image has only 2 dimensions, expand it to have a third dimension
    if len(image_array.shape) == 2:
        image_array = np.expand_dims(image_array, axis=2)

    return image_array

class ImageSequence(Sequence):
    """
    A Keras-compatible data generator that loads and preprocesses images in batches.

    Parameters:
        images (list of tuples): a list of tuples containing the input and target images.
        batch_size (int): the batch size to use during training.
        shuffle (bool): whether to shuffle the data between epochs.
        target_size (int): the size to which the images should be resized (assumed to be square).
    """
    def __init__(self, images, batch_size, shuffle=True, target_size=224):
        self.images = images
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.target_size = target_size
        self.indices = np.arange(len(images))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.images[i] for i in batch_indices]
        inputs = np.array([preprocess_input(x[0]) for x in batch])
        targets = np.array([preprocess_input(x[1]) for x in batch])
        return inputs, targets

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

from pathlib import Path

def load_images_from_folder(folder_path, target_size):
    """
    Load images from a folder and resize them to the specified target size.

    Parameters:
        folder_path (str or Path): the path to the folder containing the images.
        target_size (int): the size to which the images should be resized (assumed to be square).

    Returns:
        a list of preprocessed input and target images.
    """
    image_paths = sorted(glob.glob(os.path.join(folder_path, '*.jpg')))
    images = []
    for path in image_paths:
        image = load_img(path, target_size=(target_size, target_size))
        input_image = img_to_array(image)
        target_path = Path(path.replace('foggy', 'clear'))
        target_path = target_path.parent / (target_path.stem + '.jpg')
        target_image = img_to_array(load_img(target_path, target_size=(target_size, target_size)))
        images.append((input_image, target_image))
    return images


def supervised_train(foggy_folder_path, clear_folder_path, num_epochs, batch_size, target_size):
    """
    Train the DehazeNet model using supervised learning.

    Parameters:
        foggy_folder_path (str or Path): the path to the folder containing the foggy images.
        clear_folder_path (str or Path): the path to the folder containing the corresponding clear images.
        num_epochs (int): the number of epochs to train for.
        batch_size (int): the batch size to use during training.
        target_size (int): the size to which the images should be resized (assumed to be square).

    Returns:
        a Keras model object trained on the input data.
    """
    # Load and preprocess the images
    train_images = load_images_from_folder(foggy_folder_path, target_size)
    val_images = load_images_from_folder(clear_folder_path, target_size)

    # Create data generators for loading and preprocessing images in batches
    train_generator = ImageSequence(train_images, batch_size=batch_size, shuffle=True)
    val_generator = ImageSequence(val_images, batch_size=batch_size, shuffle=False)

    # Build and compile the DehazeNet model
    num_channels = 3  # Assume RGB input images
    model = build_dehazenet_model(target_size, num_channels)
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the DehazeNet model using supervised learning
    early_stopping = callbacks.EarlyStopping(patience=3, restore_best_weights=True)
    model.fit(
        train_generator,
        epochs=num_epochs,
        steps_per_epoch=len(train_images) // batch_size,
        validation_data=val_generator,
        validation_steps=len(val_images) // batch_size,
        callbacks=[early_stopping]
    )

    return model


if __name__ == '__main__':
    foggy_folder_path = Path("C:/Users/HP/Pictures/clearFoggy/Foggy/")  # Specify the folder path containing the foggy images
    clear_folder_path = Path("C:/Users/HP/Pictures/clearFoggy/Clear/")  # Specify the folder path containing the clear images

    target_size = 32  # Specify the desired target size for preprocessing

    foggy_images = load_images_from_folder(foggy_folder_path, target_size)
    clear_images = load_images_from_folder(clear_folder_path, target_size)

    print("Number of foggy images:", len(foggy_images))
    print("Number of clear images:", len(clear_images))

    # Train the DehazeNet model
    num_epochs = 10
    batch_size = 32
    model = supervised_train(foggy_folder_path, clear_folder_path, num_epochs, batch_size, target_size)

    # Evaluate the DehazeNet model on a test image (optional)
    test_image_path = "C:/Users/HP/Documents/Semester (summer)/Aim Lab Internship/Fog/foggy-001.jpg"
    test_image = Image.open(test_image_path)
    test_foggy_image = preprocess_image(test_image, target_size)
    test_foggy_image = np.expand_dims(test_foggy_image, axis=0)
    predicted_clear_image = model.predict(test_foggy_image)
    plt.imshow(test_foggy_image[0])
    plt.show()
    plt.imshow(predicted_clear_image[0])
    plt.show()

Number of foggy images: 10
Number of clear images: 10
Epoch 1/10


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.